# Data

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)
sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtest.utils import run_strategy, run_wfo


from backtesting import Backtest
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
from pandas import Timestamp
import numpy as np
import random

random.seed(42)

In [3]:
initial_cash = 10_000
margin = 1/30
commission = 7e-4

In [4]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()

groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [5]:

tickers = [symbol.path.split('\\')[1] for symbol in symbols if ('Energies_group' in symbol.path or 'Indices_group' in symbol.path)]
# tickers = ['USOILm']

print(tickers)

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2021, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    mt5.TIMEFRAME_H4,
    mt5.TIMEFRAME_H2,
    # mt5.TIMEFRAME_H1,
    # mt5.TIMEFRAME_M15,
]

symbols = get_data(tickers, intervals, date_from, date_to)

['UKOILm', 'USOILm', 'AUS200m', 'DE30m', 'FR40m', 'HK50m', 'JP225m', 'STOXX50m', 'UK100m', 'US30m', 'US500m', 'USTECm', 'XNGUSDm', 'IN50m']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
UKOILm
UKOILm
USOILm
USOILm
AUS200m
AUS200m
DE30m
DE30m
FR40m
FR40m
HK50m
HK50m
JP225m
JP225m
STOXX50m
STOXX50m
UK100m
UK100m
US30m
US30m
US500m
US500m
USTECm
USTECm
XNGUSDm
XNGUSDm
IN50m
se descarto IN50m
IN50m
se descarto IN50m


In [6]:
max_start_date = None
intervals_start_dates = {}

tickers = np.unique(list(symbols.keys())).tolist()

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates

{16388: Timestamp('2022-03-20 20:00:00+0000', tz='UTC'),
 16386: Timestamp('2022-03-20 22:00:00+0000', tz='UTC')}

In [7]:
start_date_train = Timestamp('2022-03-21 00:00:00', tz='UTC')
limit_date_train = Timestamp('2024-09-01 00:00:00', tz='UTC')

print(start_date_train)
print(limit_date_train)

2022-03-21 00:00:00+00:00
2024-09-01 00:00:00+00:00


# Analisis preliminar

In [8]:
from backtesting import Strategy
import talib as ta
from backbone.utils.general_purpose import calculate_units_size, diff_pips

class BPercent(Strategy):
    pip_value = None
    minimum_lot = None
    contract_volume = None
    minimum_units = None
        
    risk= 1
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200
    b_open_threshold = 0.95
    b_close_threshold = 0.5
    atr_multiplier = 1.8
    pip_value = 0.1

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )
        
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)
        
        self.minimum_units = self.minimum_lot * self.contract_volume

    def next(self):
        actual_close = self.data.Close[-1]
        b_percent = (actual_close - self.lower_band[-1]) / (self.upper_band[-1] - self.lower_band[-1])
        
        if self.position:
            if self.position.is_long:
                if b_percent >= self.b_close_threshold:
                    self.position.close()

            if self.position.is_short:
                if b_percent <= 1 - self.b_close_threshold:
                    self.position.close()

        else:
            if b_percent <= 1 - self.b_open_threshold and actual_close > self.sma[-1]:

                sl_price = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                units = self.minimum_units if units < self.minimum_units else units
                
                self.buy(
                    size=units,
                    # sl=sl_price
                )
                
            if b_percent >= self.b_open_threshold and actual_close < self.sma[-1]:
                sl_price = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                units = self.minimum_units if units < self.minimum_units else units

                self.sell(
                    size=units,
                    # sl=sl_price    
                )

In [9]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression

strategies = [
    BPercent
]

experiments = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    try:
        print(ticker, interval)

        df_stats = run_strategy(
            strategy=strategy,
            ticker=ticker,
            interval=interval,
            commission=commission, 
            prices=symbols[ticker][interval], 
            initial_cash=initial_cash, 
            margin=margin
        )

        performance = pd.concat([performance, df_stats])

    except:
        print(f'hubo un problema con {ticker} {interval}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


AUS200m 16388
AUS200m 16386
DE30m 16388
DE30m 16386
FR40m 16388
FR40m 16386
HK50m 16388
HK50m 16386
JP225m 16388
JP225m 16386
STOXX50m 16388
STOXX50m 16386
UK100m 16388
UK100m 16386
UKOILm 16388
UKOILm 16386
US30m 16388
US30m 16386
US500m 16388
US500m 16386
USOILm 16388
USOILm 16386
USTECm 16388
USTECm 16386
XNGUSDm 16388
XNGUSDm 16386


In [10]:

filter_performance = performance[
    (performance['return/dd']>1)
    # & (performance['stability_ratio']>0.7)
].sort_values(by=['return/dd'], ascending=[False])#.drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,BPercent,US500m,16386,0.719865,13.139803,11313.980328,8.737832,307 days 16:00:00,72.500000,0.480122,40,0.124739,12.174425,11313.980328,1038 days 02:00:00,1.503783,5.010931
0,BPercent,HK50m,16388,0.754859,7.195890,10719.589038,4.810549,202 days 08:00:00,66.666667,0.498353,18,0.470388,13.065448,10719.589038,1038 days 00:00:00,1.495856,3.646447
0,BPercent,XNGUSDm,16388,0.564415,8.566591,10856.659069,6.320077,227 days 20:00:00,77.777778,0.599092,18,1.573862,11.238532,10856.659069,894 days 00:00:00,1.355457,3.445839
0,BPercent,USTECm,16386,0.572086,16.771470,11677.146969,14.911580,303 days 08:00:00,77.272727,0.573423,44,0.289774,13.319189,11677.146969,1038 days 02:00:00,1.124728,4.012381


In [11]:
for ticker, interval in zip(portfolio, intervals):
    df_stats = run_strategy(
        strategy=strategy,
        ticker=ticker,
        interval=interval,
        commission=commission, 
        prices=symbols[ticker][interval], 
        initial_cash=initial_cash, 
        margin=1/30,
        plot=True
    )


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarn

# WFO

In [ ]:
from backtest.utils import optim_func_2

strategy = BPercent
lookback_bars = 1800
validation_bars = 100
warmup_bars = 200

all_wfo_performances = pd.DataFrame()
stats_per_symbol = {}

params = {
    'b_open_threshold' : [0.95],
    'b_close_threshold': [0.5],

}

for index, row in filter_performance.iterrows():
    
    # try:
    ticker = row.ticker
    interval = row.interval
    
    print(ticker, interval)
    
    prices = symbols[ticker][interval]

    wfo_stats, df_stats = run_wfo(
        strategy=strategy,
        ticker=ticker,
        interval=interval,
        prices=prices,
        initial_cash=initial_cash,
        commission=commission,
        margin=margin,
        optim_func=optim_func_2,
        params=params,
        lookback_bars=lookback_bars,
        warmup_bars=warmup_bars,
        validation_bars=validation_bars
    )
    
    if ticker not in stats_per_symbol.keys():
        stats_per_symbol[ticker] = {}

    stats_per_symbol[ticker][interval] = wfo_stats

    all_wfo_performances = pd.concat([all_wfo_performances, df_stats])
    
    # except:
    #     print(f'No se pudo ejecutar para el ticker {ticker}')

all_wfo_performances['return/dd'] = all_wfo_performances['return'] / -all_wfo_performances['drawdown']
all_wfo_performances['drawdown'] = -all_wfo_performances['drawdown']
all_wfo_performances['custom_metric'] = (all_wfo_performances['return'] / (1 + all_wfo_performances.drawdown)) * np.log(1 + all_wfo_performances.trades)

all_wfo_performances.drawdown_duration = pd.to_timedelta(all_wfo_performances.drawdown_duration)
all_wfo_performances.drawdown_duration = all_wfo_performances.drawdown_duration.dt.days

all_wfo_performances.sort_values(by='return/dd', ascending=False)

<!-- # Montecarlo -->

In [ ]:
filtered_wfo_performance = all_wfo_performances[
    (all_wfo_performances['stability_ratio'] > 0.7)
].sort_values(by='return/dd', ascending=False)

filtered_wfo_performance

In [14]:
# stats_per_symbol['UKOILm'][16386]['_trades']

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from backtest.utils import montecarlo_statistics_simulation

# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []
montecarlo_simulations = {}

all_drawdowns = pd.DataFrame()
all_returns = pd.DataFrame()

for index, row in filtered_wfo_performance.iterrows():
    ticker = row.ticker
    interval = row.interval
    
    print(f"Procesando ticker: {ticker}")
    trades_history = stats_per_symbol[ticker][interval]._trades
    eq_curve = stats_per_symbol[ticker][interval]._equity_curve
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, synthetic_drawdown_curve, synthetic_return_curve = montecarlo_statistics_simulation(
        equity_curve=eq_curve,
        trade_history=trades_history, 
        n_simulations=100_000, 
        initial_equity=initial_cash, 
        threshold_ruin=0.8, 
        return_raw_curves=True,
        percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
    )
    
    montecarlo_simulations[ticker] = mc
    
    synthetic_drawdown_curve['ticker'] = ticker
    synthetic_return_curve['ticker'] = ticker
    
    all_drawdowns = pd.concat([all_drawdowns, synthetic_drawdown_curve])
    all_returns = pd.concat([all_returns, synthetic_return_curve])

In [ ]:
dd_df = pd.DataFrame()
returns_df = pd.DataFrame()

for ticker, mc in montecarlo_simulations.items():
    mc = mc.rename(
        columns={
            'Drawdown (%)': f'drawdown_{ticker}',
            'Final Return (%)': f'return_{ticker}',
        }
    )

    if dd_df.empty:
        dd_df = mc[[f'drawdown_{ticker}']]
    
    else:
        dd_df = pd.merge(
            dd_df,
            mc[[f'drawdown_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
    if returns_df.empty:
        returns_df = mc[[f'return_{ticker}']]
    
    else:
        returns_df = pd.merge(
            returns_df,
            mc[[f'return_{ticker}']],
            left_index=True,
            right_index=True            
        )
        
display(dd_df)
display(returns_df)


In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 18))
sns.boxplot(data=all_drawdowns, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = all_drawdowns["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = all_drawdowns["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Generar el gráfico
plt.figure(figsize=(12, 20))
sns.boxplot(data=all_returns, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = all_returns["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = all_returns["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()
